# Processamento textual

Este notebook foi execultado utilizando a GPU do Google Colaboratory. Isso porque a base de dados é muito grande e os algoritmos de pré-processamento são muito pesados para rodar na minha máquina, podendo demorar dias para sua execução. Os dados textuais pré-processados foram salvos em um dataset para facilitar as futuras analises, uma vez que poderia demorar muito tempo para rodá-los novamente. Além disso, por a base de dados ser muito grande, só utilizei os primeiros 50 mil do dataset.

### Google Colab

A celula abaixo só deve ser executada se o notebook estiver rodando no Google Colab. Os códigos abaixo são necessários para possibilitar a execução deste notebook no colab.

In [1]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

!python3 -m spacy download pt_core_news_sm
!pip install gensim==4.2.0

2023-02-28 12:39:06.296245: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-28 12:39:07.917784: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-28 12:39:07.917983: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-28 12:39:07.918014: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

In [2]:
#Para acessar o Drive e armazenar o dataset pré-processado
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Análise dos dados do dataset

In [3]:
import pandas as pd
import re
import nltk
from nltk import word_tokenize, download
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('rslp')
from nltk.stem import RSLPStemmer
import spacy
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import numpy as np
download('punkt')
download('stopwords')
download('wordnet')
import pickle

pd.set_option("display.max_rows", 5)
pd.set_option('display.max_columns',100)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [6]:
#carrega os dados do drive
df = pd.read_csv(r"drive/My Drive/stefane/neoway/dataset/B2W-Reviews01.csv")
display(df.info(),df.sample())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132373 entries, 0 to 132372
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             132373 non-null  int64  
 1   submission_date        132373 non-null  object 
 2   reviewer_id            132373 non-null  object 
 3   product_id             132373 non-null  object 
 4   product_name           132289 non-null  object 
 5   product_brand          40982 non-null   object 
 6   site_category_lv1      132367 non-null  object 
 7   site_category_lv2      128360 non-null  object 
 8   review_title           132071 non-null  object 
 9   overall_rating         132373 non-null  int64  
 10  recommend_to_a_friend  132355 non-null  object 
 11  review_text            129098 non-null  object 
 12  reviewer_birth_year    126389 non-null  float64
 13  reviewer_gender        128237 non-null  object 
 14  reviewer_state         128382 non-nu

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


None

,Unnamed: 0,submission_date,reviewer_id,product_id,product_name,product_brand,site_category_lv1,site_category_lv2,review_title,overall_rating,recommend_to_a_friend,review_text,reviewer_birth_year,reviewer_gender,reviewer_state
2502,2502,2018-01-03 03:39:39,5fae3261264c908c9ba14d35146ce688658b9d1acf37f0...,131245964,Smartphone Asus Zenfone Go LTE Dual Chip Andro...,NaN,Celulares e Smartphones,Smartphone,Sem produto,1,No,Não chegou nada prazo de entrega ultrapassado ...,1989.0,F,SP


Apaga dados nulos e duplicados

In [7]:
df.dropna(subset=['review_title', 'overall_rating','recommend_to_a_friend','review_text','reviewer_birth_year'],inplace=True)
df['reviewer_birth_year'] = df['reviewer_birth_year'].astype('int')
df['recommend_to_a_friend_cat'] = df['recommend_to_a_friend'].map({'Yes':1,'No':0})
df.drop_duplicates(inplace=True)
df = df.reset_index()

## NLP

##### Remoção de caracteres especiais, numeros e pontuações

In [8]:
def remove_special_characters(text):
    """Remove os caracteres especiais de uma frase

    Args:
        text (String): Frase para remoção de seus caracteres especiais
    Returns:
        String: Frase após o pré-processamento
    """    
    characters_1 = re.compile("[$.;:!\'?@,\"()\[\]]")
    characters_2 = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
    characters_3 = re.compile('[0-9]+')

    text = characters_1.sub("", text)
    text = characters_2.sub(" ", text.lower())
    text = characters_3.sub("", text)
    
    return text

##### Cria os tokens e remove as stop words

In [9]:
def token_and_remove_sw(text):
    """Remove as stop words de uma frase e tokeniza a frase. 

    Args:
        text (String): Frase para tokenização e remoção de stop words

    Returns:
        list: Uma lista contendo todos os tokens da frase
    """
    text = remove_special_characters(text)
    text_tokens = word_tokenize(text, language='portuguese') 
    stop_words = set(stopwords.words('portuguese'))
    text_without_sw = [token for token in text_tokens if token not in stop_words]
    return text_without_sw 

##### Processamento textual usando stemmatização do nltk

In [10]:
def stemmize(text):
    """Realiza o pré-processamento de uma frase, realizando remoção de caracteres especiais e de stop words,
       faz também a tokenização e stemização da frase. 

    Args:
        text (list): A frase que deve ser pré-processada

    Returns:
        list: Uma lista com os tokens stemizados
    """
    stemmer = RSLPStemmer()
    text = token_and_remove_sw(text)
    text_stem = [stemmer.stem(token) for token in text]
    return text_stem

##### Processamento textual utilizando a lematização do Spacy

In [11]:
def lemmatize(text):
    """Realiza o pré-processamento de uma frase, realizando remoção de caracteres especiais e de stop words,
       faz também a tokenização e lematização da frase. 

    Args:
        text (list): A frase que deve ser pré-processada

    Returns:
        list: Uma lista com os tokens lematizados
    """
    nlp = spacy.load('pt_core_news_sm')
    text = token_and_remove_sw(text)
    doc = nlp(str(text))
    text_lemma = [token.lemma_ for token in doc 
                  if token.pos_ not in ['PUNCT','PROPN','ADP']]
    return text_lemma

##### Processamento textual utilizando a stemmatização do nltk e lematização do Spacy

In [12]:
def lemmatize_and_stemmize(text):
    """Realiza o pré-processamento de uma frase, realizando remoção de caracteres especiais e de stop words,
       faz também a tokenização, lematização stemização da frase. 

    Args:
        text (list): A frase que deve ser pré-processada

    Returns:
        list: Uma lista com os tokens lematizados e stemizados
    """
    nlp = spacy.load('pt_core_news_sm')
    stemmer = RSLPStemmer()
    text = token_and_remove_sw(text)
    doc = nlp(str(text))
    text_lemma_stem = [stemmer.stem(token.lemma_) for token in doc 
                       if token.pos_ not in ['PUNCT','PROPN','ADP']]
    return text_lemma_stem

#### Salva os dados pré-processados no dataset

In [15]:
data = df[0:5]
data['title_stem'] = data['review_title'].apply(stemmize)
data['text_stem'] = data['review_text'].apply(stemmize)

data['title_lemma'] = data['review_title'].apply(lemmatize)
data['text_lemma'] = data['review_text'].apply(lemmatize)

data['title_lemma_stem'] = data['review_title'].apply(lemmatize_and_stemmize)
data['text_lemma_stem'] = data['review_text'].apply(lemmatize_and_stemmize)
data.to_csv(r"drive/My Drive/stefane/neoway/dataset/B2W-Reviews01_edit.csv")

<ipython-input-15-967e5535ca74>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['title_stem'] = data['review_title'].apply(stemmize)
<ipython-input-15-967e5535ca74>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text_stem'] = data['review_text'].apply(stemmize)
<ipython-input-15-967e5535ca74>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas